# Jobs scraper
My intention with this project is to scrape jobs data from the web, and pick out trends, keywords etc, to help me optimise my job applications and CV for the jobs that are actually available

# Resources
- https://github.com/spinlud/py-linkedin-jobs-scraper existing python tool for scraping jobs data from linkedin.
- 



In [18]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [19]:
base_url = 'https://www.linkedin.com/jobs/search?keywords=Data%20Science&location=Glasgow%2C%20Scotland%2C%20United%20Kingdom&geoId=102681496&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0'


In [24]:
page = requests.get(base_url)

In [25]:
page

<Response [200]>

In [28]:
soup = BeautifulSoup(page.content, 'html.parser')

In [30]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="d_jobs_guest_search" name="pageKey"/>
  <meta content="urlType=jserp_custom;emptyResult=false" name="linkedin:pageTag"/>
  <meta content="en_US" name="locale"/>
  <meta data-app-version="2.0.443" data-browser-id="11185eff-fc64-478d-84bf-74e2c53a136b" data-call-tree-id="0z6BHkGGgxbwNKNmJysAAA==" data-multiproduct-name="jobs-guest-frontend" data-service-name="jobs-guest-frontend" id="config"/>
  <meta id="google-analytics-config"/>
  <link href="https://uk.linkedin.com/jobs/data-science-jobs-glasgow" rel="canonical"/>
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <link href="https://static-exp1.licdn.com/sc/h/al2o9zrvru7aqj8e1x2rzsrca" rel="icon"/>
  <script>
   function getDfd() {let yFn,nFn;const p=new Promise(function(y, n){yFn=y;nFn=n;});p.resolve=yFn;p.reject=nFn;return p;}
            window.lazyloader = getDfd();
            window.tracking = getDfd();
            window.impressionTracking = getDfd();
           

In [32]:
job_list = soup.find(class_='jobs-search__results-list')
print(job_list.prettify())

<ul class="jobs-search__results-list">
 <!-- -->
 <li>
  <div class="base-card base-card--link base-search-card base-search-card--link job-search-card" data-column="1" data-entity-urn="urn:li:jobPosting:2486553373" data-row="1" data-search-id="tpEfJgnHTZoRDsljjNOWXg==" data-tracking-id="bacergLHZtWK+FGCJ+5gCw==">
   <a class="base-card__full-link" data-tracking-client-ingraph="" data-tracking-control-name="public_jobs_jserp-result_search-card" data-tracking-will-navigate="" href="https://uk.linkedin.com/jobs/view/data-science-consultant-at-capgemini-invent-2486553373?refId=tpEfJgnHTZoRDsljjNOWXg%3D%3D&amp;trackingId=bacergLHZtWK%2BFGCJ%2B5gCw%3D%3D&amp;position=1&amp;pageNum=0&amp;trk=public_jobs_jserp-result_search-card">
    <span class="screen-reader-text">
     Data Science Consultant
    </span>
   </a>
   <div class="search-entity-media">
    <img alt="" class="artdeco-entity-image artdeco-entity-image--square-4" data-delayed-url="https://media-exp1.licdn.com/dms/image/C4D0BAQHtB

In [33]:
card = job_list.find(class_='base_card')
card

# Selenium approach
I realised I can't get the job description from the HTML, so need a web-browser based scraping method. Using Selenium